In [0]:
%pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.0 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
spark.conf.set(
    "fs.azure.account.key.mystore.dfs.core.windows.net",
    "XXXXXXXXXXXXXXXXXXXXXXX"
)

In [0]:
from pyspark.sql.functions import expr
df = spark.read.csv("abfss://rawdata@mystore.dfs.core.windows.net/data_29_04.csv", inferSchema= True, header= True)
df.show()

+----+----------+--------------------+-------+--------+----+----+------+-----------+
|SlNo|      Date|              Status|  Style|Category|Size| Qty|Amount|  ShipState|
+----+----------+--------------------+-------+--------+----+----+------+-----------+
|   1|2022-04-29|             Shipped|JNE3721|   kurta| XXL|   2|   329|  RAJASTHAN|
|   2|2022-04-29|             Shipped|JNE3405|   kurta|  XL|   5|   399|      DELHI|
|   3|2022-04-29|Shipped - Deliver...|JNE3697|    NULL| XXL|   1|   458|    HARYANA|
|   4|2022-04-29|             Shipped| SET254|     Set|  XL|   7|456790|  KARNATAKA|
|   5|2022-04-29|             Shipped|JNE3795|   kurta|NULL|   1|   517| TAMIL NADU|
|   6|2022-04-29|             Shipped| SET345|     Set|NULL|NULL|   148|  KARNATAKA|
|   7|2022-04-29|             Shipped|JNE3373|   kurta|   L|   1|   376|  TELANGANA|
|   8|2022-04-29|           Cancelled| SET291|     Set|   M|   0|  NULL|MAHARASHTRA|
|   9|2022-04-29|             Shipped| SET345|     Set|   M|   4|

In [0]:
OPENAI_ENDPOINT = "https://XXXXX.openai.azure.com/"
OPENAI_KEY = "XXXXXXXXX"
from openai import AzureOpenAI

EMBEDDING_MODEL = "gpt-4o"

client = AzureOpenAI(
    api_key=OPENAI_KEY,
    azure_endpoint=OPENAI_ENDPOINT,
    api_version="2024-12-01-preview"
)

In [0]:
def run_python(code: str):
    local_vars = {}
    exec(code, globals(), local_vars)

    if "result" in local_vars:
        if hasattr(local_vars["result"], "toPandas"):
            return local_vars["result"].toPandas().to_dict()
        return local_vars["result"]

    return "Code executed successfully"


In [0]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "run_python",
            "description": "Execute Python or Spark code in Databricks",
            "parameters": {
                "type": "object",
                "properties": {
                    "code": {
                        "type": "string",
                        "description": "Executable Python/Spark code"
                    }
                },
                "required": ["code"]
            }
        }
    }
]


In [0]:
system_prompt = """
You are a senior data analytics agent running inside Databricks.

Rules:
- Use python for data access for already loaded dataframe df
- Assign final output to variable named `result`
- Do not print anything
- If asked to store results, write to Hive Metastore 
- If calculation is required, generate executable Python code
- You may store results to SQL tables if asked
"""


In [0]:
user_question = """
find and delete duplicate row except SlNo
SlNo is a unique identifier and must be excluded from duplicate checks
Calculate and impute missing value of columns by using mean, median, mode
Find the outliers in IQR method and replace the outliers with the upper limit or lower limit
Store the result in Hive Metastore table
use catalog `hive_metastore`;
hive_metastore.default.sales_dec_2022
"""

In [0]:
response = client.chat.completions.create(
    model="gpt-4o",   
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_question}
    ],
    tools=tools,
    tool_choice="auto"
)


Trace(trace_id=tr-5bac5a644158930f128022a3cb813684)

In [0]:
message = response.choices[0].message
import json
import pandas as pd
from pyspark.sql.types import FloatType, DoubleType, IntegerType,StringType, StructType, StructField

if message.tool_calls:
    for tool_call in message.tool_calls:
        if tool_call.function.name == "run_python":
            args = json.loads(tool_call.function.arguments)
            tool_result = run_python(args["code"])

            # Send execution result back to OpenAI
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_question},
                    message,
                    {
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "content": json.dumps(tool_result)
                    }
                ]
            )
else:
    pass


Trace(trace_id=tr-3269264b1866cf6fc2fd2b4b594eaa0c)

In [0]:
final_answer = response.choices[0].message.content
final_answer


'The duplicate rows (except the `SlNo` column) were removed, missing values were handled using mean, median, or mode (as applicable), and outliers were identified and replaced using the IQR method. \n\nThe final cleaned dataset has been stored in the Hive Metastore table `hive_metastore.default.sales_dec_2022`.'

In [0]:
display(tool_result)


'Code executed successfully'

In [0]:
%sql
use catalog `hive_metastore`; select * from `default`.`sales_dec_2022` limit 100;

SlNo,Date,Status,Style,Category,Size,Qty,Amount,ShipState
8,2022-04-29,Cancelled,SET291,Set,M,0.0,799.0,MAHARASHTRA
7,2022-04-29,Shipped,JNE3373,kurta,L,1.0,376.0,TELANGANA
2,2022-04-29,Shipped,JNE3405,kurta,XL,5.0,399.0,DELHI
1,2022-04-29,Shipped,JNE3721,kurta,XXL,2.0,329.0,RAJASTHAN
5,2022-04-29,Shipped,JNE3795,kurta,XL,1.0,517.0,TAMIL NADU
4,2022-04-29,Shipped,SET254,Set,XL,7.0,799.0,KARNATAKA
6,2022-04-29,Shipped,SET345,Set,XL,2.0,148.0,KARNATAKA
9,2022-04-29,Shipped,SET345,Set,M,4.0,148.0,KARNATAKA
3,2022-04-29,Shipped - Delivered to Buyer,JNE3697,Set,XXL,1.0,458.0,HARYANA
